In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

np.random.seed(143)

In [ ]:
stock_price = pd.read_csv("stock_data_2_yrs_daywise.csv")
stock_price.set_index('Date', inplace = True)

returns = (stock_price - stock_price.shift(1))/stock_price.shift(1)
returns.dropna(inplace = True)
returns.head()

,AdaniPorts,Ambuja,Apollo,AxisBank,Bajaj,BoB,Bata,BlueStar,Camlin,Castrol,...,SteelAuth,sunPharma,TataComm,TataSteel,Titan,Ultramarine,Wipro,Wonderla,YesBank,Nilkamal
Date,,,,,,,,,,,,,,,,,,,,,
01-Sep-23,-0.027492,-0.018092,0.007811,-0.018739,0.002547,-0.019895,-0.001284,-0.004227,-0.000662,0.019359,...,-0.066243,0.001982,-0.037178,-0.042685,-0.001284,-0.001129,-0.026376,0.018740,-0.030460,0.014803
31-Aug-23,0.029038,0.039667,0.012918,0.009138,-0.030779,0.024038,-0.010797,0.003976,0.006623,-0.012086,...,0.006699,-0.001079,0.002812,-0.005270,-0.010797,-0.037288,0.002463,0.002290,-0.000593,0.019937
30-Aug-23,0.030337,-0.002032,0.004464,-0.001831,-0.015114,0.000261,0.000195,-0.006712,-0.030592,0.000699,...,-0.033381,0.012511,-0.007718,-0.020782,0.000195,0.017488,0.004913,0.008350,0.002966,-0.057087
29-Aug-23,-0.022914,-0.007691,-0.005333,0.010091,0.005899,-0.007823,-0.009419,-0.013447,-0.011876,0.014321,...,-0.012527,-0.001244,0.023641,-0.013733,-0.009419,0.003461,0.000000,-0.009843,-0.005914,-0.005775
28-Aug-23,0.006621,-0.005471,-0.006255,-0.008073,-0.007937,-0.001051,0.005590,-0.035616,-0.016140,-0.000344,...,-0.020571,-0.010413,-0.006478,-0.013080,0.005590,-0.062651,0.006845,-0.001815,0.005949,-0.009894


In [ ]:
from scipy.optimize import minimize
r = np.array(returns)

exp_returns = r.mean(axis = 0)
std_dev = r.std(axis = 0)
cov_matrix = np.array(returns.cov())

In [ ]:
def VarRisk_minimization(shortsell = True):
  def objective1(xi):
    return np.dot(xi.T, np.dot(cov_matrix, xi))
  def objective2(xi):
    return np.dot(xi.T, exp_returns).sum()
  objective = lambda xi: 0.5*objective1(xi) - 0.5*objective2(xi) #weighted sum = 0.5
  initial_weights = [1/50]*50
  constraints = [
      {'type': 'eq', 'fun': lambda w: sum(w) - 1, 'args': ()},
      # {'type': 'ineq', 'fun': lambda w: sum(w*exp_returns) - target_return}
  ]
  if not shortsell:
    bounds = [(0., None)]*(len(initial_weights))
    result = minimize(objective, initial_weights, constraints = constraints, bounds = bounds)
  else:
    result = minimize(objective, initial_weights, constraints = constraints)
  return result.x


In [ ]:
def risk_return(weights):
  risk = np.dot(weights.T, np.dot(cov_matrix, weights))
  returns = np.dot(weights.T, exp_returns)
  return risk, returns

In [ ]:
results = VarRisk_minimization(True)
total_capital = 100000
investment = pd.DataFrame(results*total_capital)

cols = returns.columns
investment.set_index(cols, inplace = True)

In [ ]:
investment['Investment'] = investment[0]
investment.drop([0],axis = 1, inplace = True)
investment.round(3)

,Investment
AdaniPorts,94074.387
Ambuja,74141.319
Apollo,-128057.386
AxisBank,-28097.387
Bajaj,15184.184
BoB,-130552.271
Bata,-107578.120
BlueStar,44477.798
Camlin,-90574.375
Castrol,125427.460


In [ ]:
w1 = VarRisk_minimization(True)
risk_return(w1)

(0.016789432839753395, 0.033040269595241735)

In [ ]:
def MADRisk_minimization(shortsell = True):
  def objective1(xi):
    return (np.abs(((r - exp_returns)*xi)).sum(axis = 1)).sum()/r.shape[0]
  def objective2(xi):
    return np.dot(xi.T, exp_returns).sum()
  objective = lambda xi: 0.5*objective1(xi) - 0.5*objective2(xi) #weighted sum = 0.5
  initial_weights = [1/50]*50
  constraints = [
      {'type': 'eq', 'fun': lambda w: sum(w) - 1, 'args': ()},
      # {'type': 'ineq', 'fun': lambda w: sum(exp_returns*w) - target_return}
  ]
  if not shortsell:
    bounds = [(0., None)]*(len(initial_weights))
    result = minimize(objective, initial_weights, constraints = constraints, bounds = bounds)
  else:
    result = minimize(objective, initial_weights, constraints = constraints)
  return result.x

In [ ]:
result_MAD = MADRisk_minimization(True)
total_capital = 100000
investment = pd.DataFrame(result_MAD*total_capital)

cols = returns.columns
investment.set_index(cols, inplace = True)

investment['Investment'] = investment[0]
investment.drop([0],axis = 1, inplace = True)
investment.round(3)

,Investment
AdaniPorts,-42.311
Ambuja,44.106
Apollo,-18.464
AxisBank,5280.937
Bajaj,12.711
BoB,38.748
Bata,12.592
BlueStar,51.020
Camlin,38.110
Castrol,7369.935


In [ ]:
w2 = MADRisk_minimization(True)
risk_return(w2)

(8.380932175360959e-05, -0.00013790756031195178)

In [ ]:
def MMRisk_minimization(shortsell = True):
  def objective1(xi):
    return np.max(np.abs(xi - exp_returns))
  def objective2(xi):
    return np.dot(xi.T, exp_returns).sum()
  objective = lambda xi: 0.5*objective1(xi) - 0.5*objective2(xi) #weighted sum = 0.5
  initial_weights = [1/50]*50
  constraints = [
      {'type': 'eq', 'fun': lambda w: sum(w) - 1, 'args': ()},
      # {'type': 'ineq', 'fun': lambda w: sum(w*exp_returns) - target_return}
  ]
  if not shortsell:
    bounds = [(0., None)]*(len(initial_weights))
    result = minimize(objective, initial_weights, constraints = constraints, bounds = bounds)
  else:
    result = minimize(objective, initial_weights, constraints = constraints)
  return result.x

In [ ]:
result_MM = MMRisk_minimization(True)
total_capital = 100000
investment = pd.DataFrame(result_MM*total_capital)

cols = returns.columns
investment.set_index(cols, inplace = True)

investment['Investment'] = investment[0]
investment.drop([0],axis = 1, inplace = True)
investment.round(3)

,Investment
AdaniPorts,2103.194
Ambuja,2093.926
Apollo,1997.462
AxisBank,2053.056
Bajaj,2102.790
BoB,1890.451
Bata,2003.592
BlueStar,2107.740
Camlin,1949.262
Castrol,2103.487


In [ ]:
w3 = MMRisk_minimization(True)
risk_return(w3)

(0.0001857579506295628, 0.00022352436848140566)